# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,whitehorse,60.7161,-135.0538,24.45,49,75,2.24,CA,1690417368
1,1,west island,-12.1568,96.8225,23.99,83,75,10.80,CC,1690417536
2,2,waitangi,-43.9535,-176.5597,9.34,86,92,1.34,NZ,1690417531
3,3,impfondo,1.6381,18.0667,22.21,84,82,0.82,CG,1690417883
4,4,odesskoye,54.2111,72.9651,18.68,82,98,2.30,RU,1690417883
...,...,...,...,...,...,...,...,...,...,...
563,563,mount pleasant,32.7941,-79.8626,29.44,79,0,2.24,US,1690417425
564,564,baker city,44.7749,-117.8344,29.84,26,0,7.72,US,1690417971
565,565,hafar al-batin,28.4328,45.9708,33.32,17,0,2.57,SA,1690417980
566,566,castiglione del lago,43.1271,12.0453,19.00,82,1,1.28,IT,1690417980


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot_1

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[(city_data_df['Max Temp']>21) & 
                               (city_data_df['Max Temp']<27) &
                               (city_data_df["Wind Speed"] < 4.5) &
                               (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
171,171,carthage,36.8611,10.3316,24.74,65,0,3.60,TN,1690417749
278,278,kone,-21.0595,164.8658,24.10,39,0,3.97,NC,1690417930
306,306,taman',45.2117,36.7161,24.14,67,0,3.33,RU,1690417934
317,317,sarykemer,43.0000,71.5000,24.71,27,0,3.45,KZ,1690417937
329,329,brooks,50.5834,-111.8851,26.99,22,0,2.57,CA,1690417381
340,340,nantucket,41.2835,-70.0995,24.44,94,0,3.60,US,1690417710
381,381,nica,43.7031,7.2661,22.89,45,0,2.06,FR,1690417949
382,382,yarmouth,41.7057,-70.2286,25.00,82,0,4.12,US,1690417594
425,425,al bayda',32.7627,21.7551,25.58,42,0,3.85,LY,1690417957
459,459,gokcebey,41.3058,32.1423,24.58,43,0,3.46,TR,1690417962


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
171,carthage,TN,36.8611,10.3316,65,
278,kone,NC,-21.0595,164.8658,39,
306,taman',RU,45.2117,36.7161,67,
317,sarykemer,KZ,43.0000,71.5000,27,
329,brooks,CA,50.5834,-111.8851,22,
340,nantucket,US,41.2835,-70.0995,94,
381,nica,FR,43.7031,7.2661,45,
382,yarmouth,US,41.7057,-70.2286,82,
425,al bayda',LY,32.7627,21.7551,42,
459,gokcebey,TR,41.3058,32.1423,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
carthage - nearest hotel: Hôtel Amilcar
kone - nearest hotel: Pacifik Appartels
taman' - nearest hotel: Волна
sarykemer - nearest hotel: No hotel found
brooks - nearest hotel: No hotel found
nantucket - nearest hotel: No hotel found
nica - nearest hotel: Hôtel du Centre
yarmouth - nearest hotel: The Inn At Cape Cod
al bayda' - nearest hotel: فندق الهديل
gokcebey - nearest hotel: No hotel found
bni bouayach - nearest hotel: No hotel found
gargalianoi - nearest hotel: Messinian Horizons
igarka - nearest hotel: Заполярье
zaragoza - nearest hotel: Hotel Pilar Plaza
paveh - nearest hotel: هتل ارم
chelghoum el aid - nearest hotel: Rhumel Hotel
chipiona - nearest hotel: Hotel Chipiona


,City,Country,Lat,Lng,Humidity,Hotel Name
171,carthage,TN,36.8611,10.3316,65,Hôtel Amilcar
278,kone,NC,-21.0595,164.8658,39,Pacifik Appartels
306,taman',RU,45.2117,36.7161,67,Волна
317,sarykemer,KZ,43.0000,71.5000,27,No hotel found
329,brooks,CA,50.5834,-111.8851,22,No hotel found
340,nantucket,US,41.2835,-70.0995,94,No hotel found
381,nica,FR,43.7031,7.2661,45,Hôtel du Centre
382,yarmouth,US,41.7057,-70.2286,82,The Inn At Cape Cod
425,al bayda',LY,32.7627,21.7551,42,فندق الهديل
459,gokcebey,TR,41.3058,32.1423,43,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)